# ex08_find_rbbh.ipynb - Finding reciprocal best BLAST hits <img src="data/JHI_STRAP_Web.png" style="width: 150px; float: right;"> 

In [ ]:
%pylab inline

## Overview

The purpose of this exercise is to carry out each stage in a reciprocal best BLAST hit (RBBH) comparison, to find putative orthologues between the protein complements of a set of bacteria. This involves:


1. **Conducting BLASTP comparisons** between sets of proteins
2. **Processing the output** to identify reciprocal best BLAST hits
3. **Visualisation and interpretation** of the output

## Importing Python code

Some code is exercise-specific, and will be imported from the local `bs32010` module in this directory.

In [ ]:
from bs32010.ex08 import *  # Local functions and data

## 1. Conducting BLASTP comparisons

For demonstration purposes, three chromosomal protein complements of related bacteria are provided in the `find_rbbh/data` directory:

* `NC_004547.faa`: *Pectobacterium atrosepticum* SCRI1043
* `NC_013421.faa`: *Pectobacterium wasabiae* WPP163
* `NC_010694.faa`: *Erwinia tasmaniensis* ET1_99

BLASTP comparisons of each protein sequence file were conducted against both of the other protein sequence files:

* `NC_004547` vs `NC_013421`
* `NC_004547` vs `NC_010694`
* `NC_013421` vs `NC_004547`
* `NC_013421` vs `NC_010694`
* `NC_010694` vs `NC_004547`
* `NC_010694` vs `NC_013421`

using BLAST commands of the form:

```
blastp -query data/NC_004547.faa -subject data/NC_013421.faa \
       -outfmt "6 qseqid sseqid qlen slen length nident pident evalue bitscore" \
       -out output/NC_004547_vs_NC_013421.tab -max_target_seqs 1
```

These commands have been run for you already to save time (on my laptop, without parallelisation, the searches take around 15min) but, if you would like to see how they were run, please take a look at the accompanying script file `run_rbbh_blast.sh`. You can do this by entering

```
!cat run_rbbh_blast.sh
```

in an iPython cell (see below). This results in the production of six BLAST output files.

In [ ]:
# Use this cell to drop out to the shell

## 2. Processing BLASTP output, and identifying reciprocal best BLAST hits

For demonstration purposes, I chose to generate non-standard BLAST output. The BLAST command was set to find a single match for each input protein sequence, and to generate a tab-separated plain text table listing data for each match, with the following columns:

    query_id  subject_id  query_length  subject_length alignment_length  identical_sites  percentage_identity  E-value  bitscore
    
Output files are in the `data/rbbh_output` directory. You can see them by running `ls` or `tree`, e.g.

```
!tree -sh data/rbbh_output
!ls -ltrh data/rbbh_output
```

In [ ]:
# Use this cell to drop out to the shell

### Coverage and Identity

The tab-separated tabular format generated above allows us to process the BLAST output easily into a `Pandas` dataframe (similar to an `R` dataframe, which you may be familiar with). For this output, I chose to recover only the information relevant to determining whether two sequences are RBBH matches. This includes the information required to filter BLAST matches on the basis of two parameters: *coverage* and *identity*, and illustrate their relationship to E-value and bitscore.

* **identity**: This is the percentage of the BLASTP alignment that is made up of identical amino acid sequence, in the two matching sequences.
* **coverage**: There are two coverage values, one for the query sequence, and one for the subject sequence. They represent the percentage of the query (or subject) sequence that is covered by the aligned region.

These are useful parameters because they allow us to tune our BLAST output to reflect the degree of sequence similarity (and therefore presumed functional similarity) we want to have in our final RBBH set (see presentation slides).

To calculate percentage identity ($PID$) of the BLAST match directly (though this value is provided for us to use in the BLAST output), we would calculate:

$$
PID = \frac{\text{identical_sites}}{\text{alignment_length}}
$$

and to calculate query and subject coverage ($COV_{q}$ and $COV_{s}$), we find:

$$
COV_q = \frac{\text{alignment_length}}{\text{query_length}} \\
COV_s = \frac{\text{alignment_length}}{\text{subject_length}}
$$

#### ACTIVITY 1

Use the code cell below to calculate the percentage identity, query and subject sequence coverage for this BLAST match from `NC_010694_vs_NC_013421.tab` (the meaning of each column is given in the text above):

```
gi|188532486|ref|YP_001906283.1|  gi|261820109|ref|YP_003258215.1|  93  87  76  61  80.26  1e-39  127
```

In [ ]:
# Use this cell to calculate your solution to Exercise 1 above.
# the input BLAST output data line is given below
line = "gi|188532486|ref|YP_001906283.1|  gi|261820109|ref|YP_003258215.1|  93  87  76  61  80.26  1e-39  127"

### Parsing BLASTP output

We can use the Python code provided in the `bs32010.ex08` module to process BLASTP output and obtain a set of RBBH for each pairwise comparison. The code uses `Pandas` to hold the BLASTP output, and the processed RBBH in dataframes, for ease of processing. 

The function `find_rbbh()` takes four arguments. The first two arguments are the accession numbers for the genomes that are to be compared: `NC_004547`, `NC_013421` and `NC_010694`. The next two (optional) arguments are the required percentage identity of the RBBH match, and the minimum coverage of either query or subject sequence that we will accept, for a RBBH.

We can identify all the RBBH for any pair of our example genomes, using the `find_rbbh()` function. This returns three dataframes:

* `df1`: the forward direction one-way BLAST search
* `df2`: the reverse direction one-way BLAST search
* `rbbh`: the corresponding set of reciprocal best hits

By using the `head()` method of each dataframe, we can inspect the first few rows.

```python
df1, df2, rbbh = find_rbbh('NC_004547', 'NC_013421', pid=0, cov=0)
df1.head()
df2.head()
rbbh.head()
```

We do this now for the *Pectobacterium* chromosomes `NC_004547` and `NC_013421`.

**NOTE: you can experiment with different combinations of input files, and percentage identity and coverage cutoffs**

In [ ]:
# Enter code iin this cell

We can see what size each dataframe is by using its `.shape` attribute, and the number of rows by using `len()`. So, to compare the shapes and row counts of `df1`, `df2` and `rbbh`, we can write:

```python
print("Shapes - df1: %s, df2: %s, rbbh: %s" % (df1.shape, df2.shape, rbbh.shape))
print("Row counts - df1: %s, df2: %s, rbbh: %s" % (len(df1), len(df2), len(rbbh)))
```

In [ ]:
# Enter code in this cell

Both methods tell us that, with `pid` and `cov` set to their default values of zero, using `NC_004547` as a one-way query against `NC_013421` gives 4466 matches, while the reverse one-way search gives 4434 matches. 

It also tells us that the total count of RBBH from those two comparisons is 3481 (i.e. around 80% of one-way searches give rise to RBBH).

#### ACTIVITY 2

Why do you think that only around 80% of one-way best BLAST matches in our example give rise to RBBH?

We can get more summary information out of a dataframe with its `describe()` method, using:

```python
rbbh.describe()
```

In [ ]:
# Enter code in this cell

From this example we can see that, while the average RBBH percentage coverage and identity are quite high (93% and 98%), in some cases these can drop as low as 19% and 2%, respectively. Clearly, not all RBBH are equally high-quality, and we will want to filter our results when applied to real data and analyses.

#### ACTIVITY 3

The maximum values of query and subject coverage are greater than 100% - why do you think this can happen?

## 3. Visualising and interpreting RBBH output

In this section, we will use simple visualisation approaches to inspect the behaviour of the one-way and reciprocal BLAST output calculated above.

### One-way BLAST output

We can view some of the basic relationships between BLAST's reported measures of match quality, using scatterplots.

#### E-values

Often, BLAST's E-values are taken to be reliable indicators of match quality, and are often (**incorrectly**) interpreted as probability values. However, E-values are influenced by query sequence length, and the size of the database being searched against. They are not simply a reflection of alignment quality, but alignment quality *in the context of the exact search that was done*. The same alignment/match can return multiple different E-values, depending on the database in which the match was found. This makes it unreliable as a comparator between different BLAST searches

#### (Normalised) Bitscores

BLAST's E-values are calculated from the alignment bitscore, modified to reflect the query sequence length and database composition. The (normalised) bitscore itself directly reflects the sequence alignment, and is more reliable for comparison across different BLAST searches.

#### Percentage identity

BLAST reports percentage identity of the aligned region, **not** percentage identity between the query and subject sequence. Two sequences that share a common motif exactly may, in some circumstances, result in an alignment with 100% identity, even though the sequences are quite different - even if they differ over nearly all their length.

#### Percentage coverage

This is not reported directly by BLAST, and has to be derived from its output (see above). As calculated above, it reflects the proportion of the query or subject sequence covered by the BLAST alignment. It is useful therefore to distinguish between instances where a high percentage identity corresponds to a full-length protein match, and where it reflects only that a single domain is found to be in common between two proteins. Note that there are two possible coverage values: one for the query, and one for the subject.

#### Activity 4

We can inspect the relationship between E-value and bitscore for one-way BLAST hits with a scatterplot, using the function `plot_scatter()` defined in `bs32010.ex08`:

```python
plot_scatter(df1.bitscore, log10(df1.Evalue), 
             "bitscore", "log_{10}(E-value)", 
             "one-way E-value vs bitscore")
```

In [ ]:
# Enter code in this cell

The approximately linear relationship between bitscore and E-value, where large bitscore corresponds to lower E-value, is clear from the plot. However, there is a reasonable amount of deviation around that line, and the relationship breaks down more significantly as bitscore increases, and the E-value approaches the lowest value that can be represented by the computer.

#### Activity 5

How do E-value and bitscore vary with query length? Scatter plots can help, for which we can use the `plot_scatter()` function from the `bs32010.ex08` module:

```python
plot_scatter(df1.query_length, log10(df1.Evalue), 
             "query length", "log_{10}(E-value)", 
             "one-way E-value vs query length")
plot_scatter(df1.query_length, df1.bitscore, 
             "query length", "bitscore", 
             "one-way bitscore vs query length")             
```

In [ ]:
# Enter code in this cell

This throws up some interesting observations:

1. As query length increases, the 'best' (smallest) E-value that can be reported also falls.
2. Bitscores show two relationships - a population for which bitscore increases with query length, and a population for which bitscore does not increase with query length.
3. The largest query lengths correspond to the lowest E-values, but surprisingly low bitscores.

So, longer queries might make one-way matches look 'better' than they really are, if only E-values are considered.

#### Activity 6

How are query and subject sequence coverage related?

For this question, we move from 2D scatterplots to heatmaps/2D histograms. In this representation, the X and Y values define a plane, which is divided into a grid (or sometimes [hexfield](http://matplotlib.org/examples/pylab_examples/hexbin_demo.html)) of *cells*. These cells are coloured by the number of points that fill them, so that variation in the frequency of datapoints can easily be seen.

Here, we are plotting one-way BLAST hit query coverage on the x-axis, and subject coverage on the y-axis. Cells are filled with colours from the default (`jet`) colourmap. The `plot_hist2d` function defines 100 bins in each axis by default, so there are approximately 10,000 cells in the resulting plots. In this case, cells with large numbers of points are coloured red, and those with few points are coloured blue (on a log scale). Empty cells are left white.

#### ACTIVITY 7

What pattern do you expect to see from plotting subject against query coverage from one-way BLAST hits?

We can inspect this with the `plot_hist2d()` function from the `bs32010.ex08` module:

```python
plot_hist2d(df1.query_coverage, df1.subject_coverage, 
            "one-way COVq", "one-way COVs", 
            "one-way coverage comparison")
```

In [ ]:
# Enter code in this cell

The relationship between query and subject sequence coverage is a complex one:

1. The vast majority of datapoints (i.e. BLAST hits) are at or near 100% coverage of both query and subject sequences.
2. There is a population of hits with approximately linear 1:1 relationship (bottom-left to top-right) of query to subject sequence coverage.
3. There is a population of queries with ≈100% coverage, but whose subject sequence coverage varies across the full range from 15-100%
4. There is a large population of query sequences with low coverage (<60%) whose best BLAST hit covers less than 20% of the subject sequence.

#### Activity 8

What kinds of sequence alignments might explain these different populations, and what sort of sequence similarity of query and subject sequences might be associated with each?

#### Activity 9

What is the relationship between percentage identity, and coverage?

We can plot alignment identity against e.g. query sequence coverage to see whether we should trust the alignment identity scores by themselves with the `plot_hist2d()` function in the `bs32010.ex08` module:

```python
plot_hist2d(df1.query_coverage, df1.percentage_identity, 
            "query coverage", "percentage identity", 
            "one-way percentage identity vs query coverage")
```

What kind of relationship between these values do you expect to see?

In [ ]:
# Enter code in this cell

Again, we find a complex relationship. The bulk of the data is found in the upper right corner of the plot, indicating that this is a full-length match to the query sequence, and that the match has nearly 100% identity.

But there is a second population that is smeared across the lower half of the graph, where the query coverage varies widely, and the percentage identity rarely rises above 50%

#### ACTIVITY 10

What kinds of sequence alignments might explain these two populations?

Now that we know how our one-way BLAST matches behave, we can see how considering only reciprocal best BLAST matches affects our results.

### Reciprocal Best BLAST Hits

Now we can compare the relationship between our RBBHs and the one-way BLAST matches, to see how one-way BLAST searches compare to the identification of putative orthologues.

If you have been experimenting with other datasets and code, you can restore the working dataset for the activities with the code:

```python
df1, df2, rbbh = find_rbbh('NC_004547', 'NC_013421', pid=0, cov=0)
```

In [ ]:
# Enter code in this cell

We can render the same plots as we did above, and compare them to see how finding RBBH affects the kinds of BLAST alignments that are retained.

#### Example 1 

How finding RBBH affects alignment coverage.

To see this, we can plot 2D histograms of the original one-way BLAST matches, and the reciprocial BLAST matches:

```
plot_hist2d(df1.query_coverage, df1.subject_coverage, 
            "one-way COVq", "one-way COVs", 
            "one-way coverage comparison", bins=50)
plot_hist2d(rbbh.query_coverage_x, rbbh.subject_coverage_x, 
            "RBBH COVq", "RBBH COVs", 
            "RBBH coverage comparison", bins=50)
```

In [ ]:
# Enter code in this cell

By retaining only RBBH, most of the complexity of the coverage plot has been discarded. In particular, the large cluster of low COVs sequences in the lower-left of the graph, corresponding to query sequences matching only a subdomain of the subject sequence, has disappeared.

Although there are still small numbers of RBBH where coverage is low in one, other or both query and subject sequences, the matches have been very efficiently reduced to the large set in the upper-right corner, indicative of full-length matches of both query and subject sequence, which are intuitively more likely to correspond to orthologues than matches in any other region of the plot.

#### Example 2

How does finding RBBH affect the coverage and percentage identity?

Again, to see this, we can plot 2D histograms of the original one-way BLAST matches, and the reciprocial BLAST matches:

```python
plot_hist2d(df1.query_coverage, df1.percentage_identity, 
            "query coverage", "percentage identity", 
            "one-way percentage identity vs query coverage",
            bins=50)
plot_hist2d(rbbh.query_coverage_x, rbbh.percentage_identity_x, 
            "query coverage", "percentage identity", 
            "RBBH percentage identity vs query coverage",
            bins=50)
```

In [ ]:
# Enter code in this cell

We see a similar effect in the plot of RBBH alignment identity against alignment coverage. The complexity seen in the one-way BLAST match plot has all but disappeared when only RBBH are considered. This has left, almost exclusively, BLAST matches to the full query sequence, with a level of sequence identity above 60% for the alignment.

There are, again, small numbers of RBBH for which there is not a (nearly) full-length query sequence alignment, or for which the alignment identity is low. These can be removed by imposing a coverage and/or sequence identity filter, e.g. using the `pid` and `cov` arguments of the `find_rbbh()` function.

#### ACTIVITY 11

What values of `pid` and `cov` do you think might be reasonable to restrict the RBBH results further, to exclude the majority of matches between non-orthologues, and why? Example code that may be useful for your own experimentation is given below.

This code generates a new RBBH data set: `rbbh_40` (it doesn't matter that `df1` and `df2` are overwritten, here - we're only comparing RBBH). Also, two histograms are plotted - the first shows all RBBH, and the second shows the effect of the different percentage identity and coverage settings.

```python
df1, df2, rbbh = find_rbbh('NC_004547', 'NC_013421', pid=0, cov=0)       # original
df1, df2, rbbh_40 = find_rbbh('NC_004547', 'NC_013421', pid=40, cov=40)  # new
plot_hist2d(rbbh.query_coverage_x, rbbh.percentage_identity_x, 
            "query coverage", "percentage identity", 
            "RBBH percentage identity vs query coverage",
            bins=50)
plot_hist2d(rbbh_40.query_coverage_x, rbbh_40.percentage_identity_x, 
            "query coverage", "percentage identity", 
            "RBBH_40 percentage identity vs query coverage",
            bins=50)
plot_hist2d(rbbh.query_coverage_x, rbbh.subject_coverage_x, 
            "one-way COVq", "one-way COVs", 
            "RBBH coverage comparison", bins=50)
plot_hist2d(rbbh_40.query_coverage_x, rbbh_40.subject_coverage_x, 
            "RBBH COVq", "RBBH COVs", 
            "RBBH_40 coverage comparison", bins=50)
```

In [ ]:
# Enter code in this cell

## 4. Writing RBBH output to file

Now we will write the RBBH found above to a `.crunch` output file suitable for viewing in **ACT**. This job is made easier because we have collected our data in a `Pandas` dataframe. 

However, we do not yet have enough information to generate such a file, because we do not have the locations of our gene features, with respect to their source sequences. The `read_genbank()` function in the `bs32010.ex08` module allows us to get this data, and the the `write_crunch()` function lets us generate `.crunch` output:

```python
features = read_genbank("data/NC_004547.gbk", "data/NC_013421.gbk")
write_crunch(rbbh, features, filename="NC_004547_vs_NC_013421.crunch")
```

In [ ]:
# Enter code in this cell

The `.crunch` file produced should now be usable in **ACT**, to visualise the comparison between these two genomes.

#### ACTIVITY 12

Visualise and comment on the RBBH comparison between the two *Pectobacterium* genomes. There appears to have been some rearrangement/inversion of the two genomes. Propose a series of events that explain this restructuring.

#### ACTIVITY 13

Repeat this analysis for comparisons of the *Pectobacterium* and *Erwinia tasmaniensis* genomes.